In [ ]:
import folium
from folium import plugins, Map
import json

sidewalks =gpd.read_file('..data/shape/Sidewalks.shx')
sidewalks_hazards = gpd.read_file('..data/shape/Sidewalk_Hazards.shx')
map = folium.Map(location=[42.361145, -71.057083],zoom_start = 30)
data = folium.GeoJson(sidewalks['geometry'])
data.add_to(map)
map.save('maps/Sidewalks.html')
csv_file = pd.read_csv("..data/preprocessed_sidewalks_hazards.csv")
csv_file.head()

csv_file["road"] = csv_file["address"].apply(lambda x: eval(x).get("road"))
csv_file["county"] = csv_file["address"].apply(lambda x: eval(x).get("county"))
csv_file["neighbourhood"] = csv_file["address"].apply(lambda x: eval(x).get("neighbourhood"))

# base_map = Map(location=[42.361145, -71.057083],zoom_start = 30)
csv_file['x'] = csv_file['x'].astype(float)
csv_file['y'] = csv_file['y'].astype(float)

def color_cluster(index):
    if index == "Fixed pinch point <36” sidewalk width":
        return 'green'
    elif index == "Trip hazards due to tree roots":
        return 'blue'
    elif index == "Trip hazards not due to tree roots": 
        return 'red'
    else:
        return 'yellow'


#hazards= csv_file.sort_values(by=["Type"],ascending=False)

for point in range(0, len(csv_file)):
    folium.CircleMarker([csv_file.iloc[point]['y'], csv_file.iloc[point]['x']],
                        color = color_cluster(csv_file.iloc[point]['Type']), fill = True, 
                        fill_color = color_cluster(csv_file.iloc[point]['Type'])).add_to(map)

map.save('maps/sidewalks/Sidewalks_hazards.html')

In [4]:
import pandas as pd
from folium import plugins, Map, CircleMarker
from folium.plugins import HeatMap , TagFilterButton
import geopandas as gpd
import folium
import geopy
import asyncio
from shapely.geometry import Polygon, Point
from pandas.io.json import json_normalize

sidewalks = gpd.read_file('../data/shape/Sidewalks.shx')
map = folium.Map(location=[42.361145, -71.057083],zoom_start = 30)
data = folium.GeoJson(sidewalks['geometry'], name='Sidewalks')
data.add_to(map)

csv_file = pd.read_csv("../data/preprocessed_sidewalks_hazards.csv")

csv_file["suburb"] = csv_file["address"].apply(lambda x: eval(x).get("suburb"))
csv_file['x'] = csv_file['x'].astype(float)
csv_file['y'] = csv_file['y'].astype(float)

def color_cluster(index):
    if index == "Fixed pinch point <36” sidewalk width":
        return 'green'
    elif index == "Trip hazards due to tree roots":
        return 'blue'
    elif index == "Trip hazards not due to tree roots": 
        return 'red'
    else:
        return 'yellow'

green_layer = folium.FeatureGroup(name='Fixed pinch point <36” sidewalk width')
blue_layer = folium.FeatureGroup(name='Trip hazards due to tree roots')
red_layer = folium.FeatureGroup(name='Trip hazards not due to tree roots')

# Create a dictionary to store suburb layers
suburb_layers = {}

# Loop through the hazards dataframe and add circle markers to corresponding layers
for point in range(0, len(csv_file)):
    suburb = csv_file.iloc[point]['suburb']
    if suburb == None:
        suburb = "Chinatown-Bay Village"
    temp = folium.CircleMarker([csv_file.iloc[point]['y'], csv_file.iloc[point]['x']],
                        color = color_cluster(csv_file.iloc[point]['Type']), fill = True, 
                        fill_color = color_cluster(csv_file.iloc[point]['Type'])).add_to(
                    suburb_layers.setdefault(suburb, folium.FeatureGroup(name=suburb, overlay=True)))


for suburb, layer in suburb_layers.items():
    layer.add_to(map)

# Add a layer control to the map to toggle between different layers
folium.LayerControl().add_to(map)




# Add TagFilterButton to the map to filter points by colors
# tag_filter_button = TagFilterButton(data=['green', 'blue', 'red'],
#                                     filter_on=True, filter_text='Filter by Color')
# tag_filter_button.add_to(map)


map.save("maps/Sidewalks_hazards_neighbors.html")





In [ ]:
import pandas as pd
from folium import plugins, Map, CircleMarker
from folium.plugins import HeatMap , TagFilterButton
import geopandas as gpd
import folium
import geopy
import asyncio
from shapely.geometry import Polygon, Point
from pandas.io.json import json_normalize

sidewalks = gpd.read_file('../data/shape/Sidewalks.shx')
map = folium.Map(location=[42.361145, -71.057083],zoom_start = 30)
data = folium.GeoJson(sidewalks['geometry'], name='Sidewalks')
data.add_to(map)

csv_file = pd.read_csv("../data/preprocessed_sidewalks_hazards.csv")

csv_file["suburb"] = csv_file["address"].apply(lambda x: eval(x).get("suburb"))
csv_file['x'] = csv_file['x'].astype(float)
csv_file['y'] = csv_file['y'].astype(float)

def color_cluster(index):
    if index == "Fixed pinch point <36” sidewalk width":
        return 'green'
    elif index == "Trip hazards due to tree roots":
        return 'blue'
    elif index == "Trip hazards not due to tree roots": 
        return 'red'
    else:
        return 'yellow'

green_layer = folium.FeatureGroup(name='Fixed pinch point <36” sidewalk width')
blue_layer = folium.FeatureGroup(name='Trip hazards due to tree roots')
red_layer = folium.FeatureGroup(name='Trip hazards not due to tree roots')

# Create a dictionary to store suburb layers
suburb_layers = {}

# Loop through the hazards dataframe and add circle markers to corresponding layers
for point in range(0, len(csv_file)):
    suburb = csv_file.iloc[point]['suburb']
    temp = folium.CircleMarker([csv_file.iloc[point]['y'], csv_file.iloc[point]['x']],
                        color = color_cluster(csv_file.iloc[point]['Type']), fill = True, 
                        fill_color = color_cluster(csv_file.iloc[point]['Type'])).add_to(
                    green_layer if csv_file.iloc[point]['Type'] == 'Fixed pinch point <36” sidewalk width' else
                    blue_layer if csv_file.iloc[point]['Type'] == 'Trip hazards due to tree roots' else
                    red_layer if csv_file.iloc[point]['Type'] == 'Trip hazards not due to tree roots' else
                    yellow_layer)

# Add the layers to the map
green_layer.add_to(map)
blue_layer.add_to(map)
red_layer.add_to(map)
yellow_layer.add_to(map)

# Add a layer control to the map to toggle between different layers
folium.LayerControl().add_to(map)




# Add TagFilterButton to the map to filter points by colors
# tag_filter_button = TagFilterButton(data=['green', 'blue', 'red'],
#                                     filter_on=True, filter_text='Filter by Color')
# tag_filter_button.add_to(map)


map.save("maps/Sidewalks_hazards.html")





In [26]:
csv_file = pd.read_csv("../data/preprocessed_sidewalks_hazards.csv")
csv_file["county"] = csv_file["address"].apply(lambda x: eval(x).get("county"))
csv_file["neighbourhood"] = csv_file["address"].apply(lambda x: eval(x).get("neighbourhood"))
csv_file["suburb"] = csv_file["address"].apply(lambda x: eval(x).get("suburb"))


NEIGHBORS = csv_file['neighbourhood'].unique()
print(NEIGHBORS)

[None 'Dock Square' 'Waterfront' 'Bulfinch Triangle' 'Financial District'
 'Church Green' 'Downtown Crossing' 'Bay Village' 'Chinatown'
 'Leather District' 'Audubon Square' 'Koreatown' 'Fields Corner'
 'Meeting House Hill' 'Gove Street' 'Mount Ida' 'Jeffries Point'
 'Dorchester Heights' 'Brighton Center' 'Highland' 'Neponset'
 'Cedar Grove' 'Oak Square' 'Fairmount' 'Eagle Hill' 'Readville'
 'East Somerville' 'Bellevue' 'Germantown' 'Orient Heights'
 'Dorchester Center' 'Mount Hope' 'Ashmont' 'Clarendon Hills'
 'Lower Mills' 'Forest Hills' 'Nubian Square' 'Bugg Village'
 'North Brighton' 'Stonybrook Village' 'Lower Allston' "Barry's Corner"
 "Packard's Corner" 'Union Square' 'Allston Heights' 'Allston Village'
 'Aberdeen' 'Cleveland Circle' 'City Point' 'Seaport' 'Roxbury Crossing'
 'Kane Square' 'Mount Bowdoin' 'Grove Hall' 'Savin Hill' 'Uphams Corner']


In [23]:
countys = csv_file['county'].unique()
print(countys)

['Suffolk County' 'Middlesex County' 'Norfolk County']


In [27]:
suburbs = csv_file['suburb'].unique()
print(suburbs)

['West End' None 'North End' 'Downtown Boston' 'Beacon Hill' 'Back Bay'
 'Fenway / Kenmore' 'Cambridgeport' 'Allston' 'Dorchester' 'East Boston'
 'South Boston' 'Roxbury' 'Jamaica Plain' 'Brighton' 'West Roxbury'
 'South End' 'Hyde Park' 'Charlestown' 'Mattapan' 'Roslindale'
 'Chestnut Hill' 'Mission Hill']


In [ ]:
js_code = '''
var layerControl = L.control.layers();

layerControl._layers.forEach(function(layer) {

  var featureGroupLayer = layer.layer;
  
  featureGroupLayer.eachLayer(function(layer) {

    if (layer instanceof L.CircleMarker) {
        layer.options.tags = layer.options.color;

    }
  });
});
'''

In [8]:
import pandas as pd
from folium import plugins, Map, CircleMarker
from folium.plugins import HeatMap , TagFilterButton
import geopandas as gpd
import folium
import geopy
import asyncio
from shapely.geometry import Polygon, Point
from pandas.io.json import json_normalize

sidewalks = gpd.read_file('../data/shape/Sidewalks.shx')
map = folium.Map(location=[42.361145, -71.057083],zoom_start = 30)
data = folium.GeoJson(sidewalks['geometry'], name='Sidewalks')
data.add_to(map)

csv_file = pd.read_csv("../data/preprocessed_sidewalks_hazards.csv")

csv_file["suburb"] = csv_file["address"].apply(lambda x: eval(x).get("suburb"))
csv_file['x'] = csv_file['x'].astype(float)
csv_file['y'] = csv_file['y'].astype(float)

def color_cluster(index):
    if index == "Fixed pinch point <36” sidewalk width":
        return 'green'
    elif index == "Trip hazards due to tree roots":
        return 'blue'
    elif index == "Trip hazards not due to tree roots": 
        return 'red'
    else:
        return 'yellow'

green_layer = folium.FeatureGroup(name='Fixed pinch point <36” sidewalk width in Allston')
blue_layer = folium.FeatureGroup(name='Trip hazards due to tree roots in Allston')
red_layer = folium.FeatureGroup(name='Trip hazards not due to tree roots in Allston')
yellow_layer = folium.FeatureGroup(name='Else')
# Create a dictionary to store suburb layers
suburb_layers = {}

# Loop through the hazards dataframe and add circle markers to corresponding layers
for point in range(0, len(csv_file)):
    suburb = csv_file.iloc[point]['suburb']
    temp = folium.CircleMarker([csv_file.iloc[point]['y'], csv_file.iloc[point]['x']],
                        color = color_cluster(csv_file.iloc[point]['Type']), fill = True, 
                        fill_color = color_cluster(csv_file.iloc[point]['Type'])).add_to(
                    green_layer if csv_file.iloc[point]['Type'] == 'Fixed pinch point <36” sidewalk width' and suburb =='Allston' else
                    blue_layer if csv_file.iloc[point]['Type'] == 'Trip hazards due to tree roots' and suburb =='Allston' else
                    red_layer if csv_file.iloc[point]['Type'] == 'Trip hazards not due to tree roots' and suburb =='Allston' else
                    yellow_layer)

# Add the layers to the map
green_layer.add_to(map)
blue_layer.add_to(map)
red_layer.add_to(map)


# Add a layer control to the map to toggle between different layers
folium.LayerControl().add_to(map)




# Add TagFilterButton to the map to filter points by colors
# tag_filter_button = TagFilterButton(data=['green', 'blue', 'red'],
#                                     filter_on=True, filter_text='Filter by Color')
# tag_filter_button.add_to(map)


map.save("maps/Sidewalks_hazards_Allston.html")



